In [1]:
# Beatriz Rodriguez
# DSCI 352 HW#3
# Question 3

In [1]:
from google.colab import files
import pandas as pd
import io
import numpy as np
import matplotlib.pyplot as plt

In [2]:
uploaded = files.upload()

Saving Titanic.csv to Titanic.csv


In [3]:
import numpy as np, random
SEED = 42
np.random.seed(SEED)
random.seed(SEED)

In [4]:
titanic = pd.read_csv(io.BytesIO(uploaded['Titanic.csv']))

In [5]:
titanic.head()

,Passenger,Class,Sex,Age,Survived
0,1,3rd,Male,Child,No
1,2,3rd,Male,Child,No
2,3,3rd,Male,Child,No
3,4,3rd,Male,Child,No
4,5,3rd,Male,Child,No


In [6]:
from sklearn.datasets import make_blobs
import matplotlib.pyplot as plt
from sklearn.datasets import make_blobs
from sklearn.model_selection import train_test_split

In [7]:
# X, y = make_blobs(n_samples=1000, n_features=2, centers=2, random_state=2023)
# print(X)
# print(X.shape)
# print(y)
# print(y.shape)

X = titanic[["Class", "Sex", "Age"]]
Y = titanic["Survived"]

In [8]:
X.iloc[:, 2].unique()

array(['Child', 'Adult'], dtype=object)

In [48]:
# classes = []
# sex = []
# age = []
# sur = []

# for i in range(len(X)):
#   if X.iloc[i, 0] == "1st":
#     classes.append(1)
#   elif X.iloc[i, 0] == "2nd":
#     classes.append(2)
#   elif X.iloc[i, 0] == "3rd":
#     classes.append(3)
#   else:
#     classes.append(0)

#   if X.iloc[i, 1] == "Male":
#     sex.append(1)
#   elif X.iloc[i, 1] == "Female":
#     sex.append(2)

#   if X.iloc[i, 2] == "Child":
#     age.append(1)
#   elif X.iloc[i, 2] == "Adult":
#     age.append(2)

#   if Y.iloc[i] == "Yes":
#     sur.append(0)
#   elif Y.iloc[i] == "No":
#     sur.append(1)


In [9]:
X_encoded = pd.get_dummies(titanic[["Class", "Sex", "Age"]], drop_first=False)
Y = titanic['Survived'].map({'No': 0, 'Yes': 1})

In [10]:
Y

,Survived
0,0
1,0
2,0
3,0
4,0
...,...
2196,1
2197,1
2198,1
2199,1


In [49]:
# Y = pd.Series(sur, name="Survived")

In [50]:
# X["Class"] = classes
# X["Sex"] = sex
# X["Age"] = age
# Y["Survived"] = sur

/tmp/ipython-input-1287275439.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X["Class"] = classes
/tmp/ipython-input-1287275439.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X["Sex"] = sex
/tmp/ipython-input-1287275439.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-ve

In [11]:
# Train and test partitions
X_train, X_test , Y_train, Y_test = train_test_split(X_encoded, Y, test_size=0.30, random_state=2023, stratify=Y)

In [23]:
class TwoNodeNN:
    def __init__(self, learning_rate, n_features):
        # Hidden layer: 2 neurons
        self.W1 = np.random.randn(2, n_features) * 0.01   # (2, d)
        self.b1 = np.zeros(2)                              # (2,)
        # Output layer: 1 neuron (binary classification)
        self.W2 = np.random.randn(1, 2) * 0.01             # (1, 2)
        self.b2 = 0.0                                      # scalar
        self.learning_rate = learning_rate

    def _sigmoid(self, x):
        x = np.clip(x, -50, 50)
        return 1.0 / (1.0 + np.exp(-x))

    def _sigmoid_deriv_from_sigmoid(self, s):
        # given s = sigmoid(z)
        return s * (1.0 - s)

    # -------- forward pass --------
    def _forward_single(self, x):
        # x: (d,)
        z1 = self.W1 @ x + self.b1            # (2,)
        a1 = self._sigmoid(z1)                # (2,)
        z2 = self.W2 @ a1 + self.b2           # (1,)
        yhat = self._sigmoid(z2)[0]           # scalar
        return z1, a1, z2.item(), yhat

    # Vectorized predict_proba for grid/batches
    def predict_proba(self, X):
        """
        X: (n, d) or (d,) or pandas DataFrame
        returns probs shape (n,) or scalar
        """
        if hasattr(X, "values"):              # DataFrame/Series
            X = X.values
        X = np.asarray(X, dtype=float)

        if X.ndim == 1:
            _, a1, _, yhat = self._forward_single(X)
            return yhat
        else:
            # Vectorized forward
            # Z1 = X @ W1.T + b1 -> (n,2)
            Z1 = X @ self.W1.T + self.b1
            A1 = self._sigmoid(Z1)
            Z2 = A1 @ self.W2.T + self.b2     # (n,1)
            Yhat = self._sigmoid(Z2).reshape(-1)
            return Yhat

    def predict(self, X, threshold=0.5):
        return (self.predict_proba(X) >= threshold).astype(int)

    # -------- backprop (MSE) for one sample --------
    def _compute_gradients(self, x, y):
        # Forward
        z1, a1, z2, yhat = self._forward_single(x)

        # Loss L = (yhat - y)^2
        dL_dyhat = 2.0 * (yhat - y)                            # scalar
        dyhat_dz2 = self._sigmoid_deriv_from_sigmoid(1/(1+np.exp(-z2)))
        # More stable: use yhat directly:
        dyhat_dz2 = yhat * (1.0 - yhat)
        delta2 = dL_dyhat * dyhat_dz2                          # scalar

        # Output layer grads
        dL_dW2 = delta2 * a1.reshape(1, -1)                    # (1,2)
        dL_db2 = delta2                                        # scalar

        # Backprop to hidden
        # delta1 = (W2^T * delta2) ⊙ sigmoid'(z1)
        sigp_z1 = self._sigmoid_deriv_from_sigmoid(a1)         # (2,)
        delta1 = (self.W2.T.reshape(2,) * delta2) * sigp_z1    # (2,)

        # Hidden layer grads
        dL_dW1 = delta1.reshape(-1, 1) @ x.reshape(1, -1)      # (2,d)
        dL_db1 = delta1                                         # (2,)

        return dL_dW1, dL_db1, dL_dW2, dL_db2

    def _update(self, dW1, db1, dW2, db2):
        lr = self.learning_rate
        self.W1 -= lr * dW1
        self.b1 -= lr * db1
        self.W2 -= lr * dW2
        self.b2 -= lr * db2

    def train(self, X, y, iterations=20000, report_every=200):
        errs = []
        n = len(y)
        for it in range(iterations):
            i = np.random.randint(n)
            dW1, db1, dW2, db2 = self._compute_gradients(X.iloc[i].values, y.iloc[i])
            self._update(dW1, db1, dW2, db2)
            if it % report_every == 0:
                yhat_all = self.predict_proba(X)
                errs.append(np.mean((yhat_all - y) ** 2))
        return errs

In [28]:
# Plot the results

learning_rate = 0.1
neural_network = TwoNodeNN(learning_rate, n_features=X_train.shape[1])
training_error = neural_network.train(X_train, Y_train, 20000)
# test_error = neural_network.train(X_test, Y_test, 1000)

In [14]:
# plt.plot(training_error)
# plt.plot(test_error)
# plt.xlabel("Iterations")
# plt.ylabel("Error for all training instances")
# plt.savefig("cumulative_error.png")
# plt.show()

In [15]:
from sklearn.metrics import accuracy_score, confusion_matrix

In [29]:
acc_train = accuracy_score(Y_train, neural_network.predict(X_train))
acc_test = accuracy_score(Y_test, neural_network.predict(X_test))
print("Training accuracy:", acc_train)
print("Test accuracy:", acc_test)

Training accuracy: 0.785064935064935
Test accuracy: 0.7821482602118003


In [30]:
cm = confusion_matrix(Y_test, neural_network.predict(X_test))
print(cm)

[[433  14]
 [130  84]]


In [ ]:
# # Plot the decision boundary. Create a mesh of x and y points. Then
# # predict the label. Then plot those with color.
# X1 = np.arange(-7, 6, 0.1)
# X2 = np.arange(-11, 11, 0.1)

# # create a rectangular grid out of two given one-dimensional arrays
# X1, X2 = np.meshgrid(X1, X2)

# X_decision = pd.DataFrame({"A": np.reshape(X1,28600), "B": np.reshape(X2,28600)})

# Z = neural_network.predict(X_decision)

# plt.scatter(x=X_decision["A"],y=X_decision["B"], marker = ".", c=Z, cmap="cool")
# plt.scatter(x=X[:,0], y=X[:,1], marker = ".", c=y, cmap="RdBu")
# plt.show()

In [31]:
class FourNodeNN:
    def __init__(self, learning_rate, n_features):
        # Hidden layer 1: 2 neurons
        self.W1 = np.random.randn(2, n_features) * 0.01   # (2, d)
        self.b1 = np.zeros(2)                              # (2,)

        # Hidden layer 2: 2 neurons
        self.W2 = np.random.randn(2, 2) * 0.01             # (2, 2)
        self.b2 = np.zeros(2)                              # (2,)

        # Output layer: 1 neuron
        self.W3 = np.random.randn(1, 2) * 0.01             # (1, 2)
        self.b3 = 0.0                                      # scalar

        self.learning_rate = learning_rate

    def _sigmoid(self, x):
        x = np.clip(x, -50, 50)
        return 1.0 / (1.0 + np.exp(-x))

    def _sigmoid_deriv_from_sigmoid(self, s):
        return s * (1.0 - s)

    # -------- forward pass --------
    def _forward_single(self, x):
        # x: (d,)
        z1 = self.W1 @ x + self.b1         # (2,)
        a1 = self._sigmoid(z1)             # (2,)

        z2 = self.W2 @ a1 + self.b2        # (2,)
        a2 = self._sigmoid(z2)             # (2,)

        z3 = self.W3 @ a2 + self.b3        # (1,)
        yhat = self._sigmoid(z3)[0]        # scalar

        return z1, a1, z2, a2, z3.item(), yhat

    def predict_proba(self, X):
        if hasattr(X, "values"):
            X = X.values
        X = np.asarray(X, dtype=float)

        if X.ndim == 1:  # single sample
            _, _, _, _, _, yhat = self._forward_single(X)
            return yhat
        else:            # batch
            Z1 = X @ self.W1.T + self.b1        # (n,2)
            A1 = self._sigmoid(Z1)
            Z2 = A1 @ self.W2.T + self.b2       # (n,2)
            A2 = self._sigmoid(Z2)
            Z3 = A2 @ self.W3.T + self.b3       # (n,1)
            Yhat = self._sigmoid(Z3).reshape(-1)
            return Yhat

    def predict(self, X, threshold=0.5):
        return (self.predict_proba(X) >= threshold).astype(int)

    # -------- backprop (MSE) --------
    def _compute_gradients(self, x, y):
        z1, a1, z2, a2, z3, yhat = self._forward_single(x)

        # Loss L = (yhat - y)^2
        dL_dyhat = 2.0 * (yhat - y)
        dyhat_dz3 = yhat * (1.0 - yhat)
        delta3 = dL_dyhat * dyhat_dz3

        # Gradients for W3, b3
        dL_dW3 = delta3 * a2.reshape(1, -1)   # (1,2)
        dL_db3 = delta3

        # Backprop to hidden2
        delta2 = (self.W3.T.reshape(2,) * delta3) * self._sigmoid_deriv_from_sigmoid(a2)
        dL_dW2 = delta2.reshape(-1,1) @ a1.reshape(1,-1)   # (2,2)
        dL_db2 = delta2

        # Backprop to hidden1
        delta1 = (self.W2.T @ delta2) * self._sigmoid_deriv_from_sigmoid(a1)   # (2,)
        dL_dW1 = delta1.reshape(-1,1) @ x.reshape(1,-1)    # (2,d)
        dL_db1 = delta1

        return dL_dW1, dL_db1, dL_dW2, dL_db2, dL_dW3, dL_db3

    def _update(self, dW1, db1, dW2, db2, dW3, db3):
        lr = self.learning_rate
        self.W1 -= lr * dW1
        self.b1 -= lr * db1
        self.W2 -= lr * dW2
        self.b2 -= lr * db2
        self.W3 -= lr * dW3
        self.b3 -= lr * db3

    def train(self, X, y, iterations=20000, report_every=200):
        errs = []
        n = len(y)

        for it in range(iterations):
            i = np.random.randint(n)
            dW1, db1, dW2, db2, dW3, db3 = self._compute_gradients(X.iloc[i].values, y.iloc[i])
            self._update(dW1, db1, dW2, db2, dW3, db3)

            if it % report_every == 0:
                yhat_all = self.predict_proba(X)
                errs.append(np.mean((yhat_all - y) ** 2))
        return errs

In [36]:
# Plot the results

learning_rate = 0.1
neural_network2 = FourNodeNN(learning_rate, n_features=X_train.shape[1])
training_error2 = neural_network2.train(X_train, Y_train, 20000)
# test_error2 = neural_network2.train(X_test, Y_test, 1000)

In [20]:
# plt.plot(training_error2)
# plt.plot(test_error2)
# plt.xlabel("Iterations")
# plt.ylabel("Error for all training instances")
# plt.savefig("cumulative_error.png")
# plt.show()

In [37]:
acc_train2 = accuracy_score(Y_train, neural_network2.predict(X_train))
acc_test2 = accuracy_score(Y_test, neural_network2.predict(X_test))
print("Training accuracy:", acc_train2)
print("Test accuracy:", acc_test2)

Training accuracy: 0.6772727272727272
Test accuracy: 0.6762481089258698


In [38]:
cm = confusion_matrix(Y_test, neural_network2.predict(X_test))
print(cm)

[[447   0]
 [214   0]]
